In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.losses import binary_crossentropy



In [2]:
color_to_label = {
    (180, 180, 180): 0,  # Gris - Végétation clairsemée
    (0, 100, 0): 1,     # Vert - Couverture arborée
    (255, 187, 34): 2,  # Orange - Broussailles
    (255, 255, 76): 3,  # Jaune - Prairie
    (240, 150, 255): 4, # Rose - Terre cultivée
    (250, 0, 0): 5,     # Rouge - Zones urbanisées
    (240, 240, 240): 6, # Blanc - Neige et glace
    (0, 100, 200): 7,   # Bleu - Plans d'eau permanents
    (0, 150, 160): 8,   # Vert-bleu - Marais herbacé
    (0, 207, 117): 9,   # Vert mangrove - Mangroves
    (250, 230, 160): 10, # Mousse - Lichens
    (0, 0, 0): 11       # Noir - Sans données
}

In [3]:
def label_to_class(label_img):
    # Convertir en RGB si nécessaire
    label_img = label_img.convert('RGB')
    label_array = np.array(label_img)
    height, width = label_array.shape[:2]
    class_map = np.zeros((height, width), dtype=int)

    for y in range(height):
        for x in range(width):
            pixel = tuple(label_array[y, x])
            class_idx = color_to_label.get(pixel, 11)  # Utilisez -1 ou un autre numéro pour les couleurs non mappées
            class_map[y, x] = class_idx

    return class_map


def load_images_and_masks(satellite_folder, mask_folder):
    satellite_images = []
    masks = []
    for filename in os.listdir(satellite_folder):
        sat_img = Image.open(os.path.join(satellite_folder, filename))
        mask_img = Image.open(os.path.join(mask_folder, filename))  # Assurez-vous que les noms correspondent

        satellite_images.append(np.array(sat_img))
        masks.append(label_to_class(mask_img))

    return np.array(satellite_images), np.array(masks)



In [4]:
path_satellite_images = 'TrainData\Esa_tile2_Sat'  # Chemin vers les images satellites
path_labelled_images = 'TrainData\Esa_tile2_resize'  # Chemin vers les images labelisées (masques)

X, Y = load_images_and_masks(path_satellite_images, path_labelled_images)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)


In [5]:

def EncoderMiniBlock(inputs, n_filters=12, dropout_prob=0.3, max_pooling=True):
    conv = Conv2D(n_filters, 
                  3,  # filter size
                  activation='relu',
                  padding='same',
                  kernel_initializer='HeNormal')(inputs)
    conv = Conv2D(n_filters, 
                  3,  # filter size
                  activation='relu',
                  padding='same',
                  kernel_initializer='HeNormal')(conv)
  
    conv = BatchNormalization()(conv, training=False)
    if dropout_prob > 0:     
        conv = tf.keras.layers.Dropout(dropout_prob)(conv)
    if max_pooling:
        next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(conv)    
    else:
        next_layer = conv
    skip_connection = conv    
    return next_layer, skip_connection

def DecoderMiniBlock(prev_layer_input, skip_layer_input, n_filters=12):
    up = Conv2DTranspose(
                 n_filters,
                 (3,3),
                 strides=(2,2),
                 padding='same')(prev_layer_input)
    merge = concatenate([up, skip_layer_input], axis=3)
    conv = Conv2D(n_filters, 
                 3,  
                 activation='relu',
                 padding='same',
                 kernel_initializer='HeNormal')(merge)
    conv = Conv2D(n_filters,
                 3, 
                 activation='relu',
                 padding='same',
                 kernel_initializer='HeNormal')(conv)
    return conv

def create_unet_model(input_shape, num_classes,n_filters=32):
    inputs = Input(input_shape)
    
  # Encoder includes multiple convolutional mini blocks with different maxpooling, dropout and filter parameters
    # Observe that the filters are increasing as we go deeper into the network which will increasse the # channels of the image 
    cblock1 = EncoderMiniBlock(inputs, n_filters,dropout_prob=0, max_pooling=True)
    cblock2 = EncoderMiniBlock(cblock1[0],n_filters*2,dropout_prob=0, max_pooling=True)
    cblock3 = EncoderMiniBlock(cblock2[0], n_filters*4,dropout_prob=0, max_pooling=True)
    cblock4 = EncoderMiniBlock(cblock3[0], n_filters*8,dropout_prob=0.3, max_pooling=True)
    cblock5 = EncoderMiniBlock(cblock4[0], n_filters*16, dropout_prob=0.3, max_pooling=False) 
    
    # Decoder includes multiple mini blocks with decreasing number of filters
    # Observe the skip connections from the encoder are given as input to the decoder
    # Recall the 2nd output of encoder block was skip connection, hence cblockn[1] is used
    ublock6 = DecoderMiniBlock(cblock5[0], cblock4[1],  n_filters * 8)
    ublock7 = DecoderMiniBlock(ublock6, cblock3[1],  n_filters * 4)
    ublock8 = DecoderMiniBlock(ublock7, cblock2[1],  n_filters * 2)
    ublock9 = DecoderMiniBlock(ublock8, cblock1[1],  n_filters)

    conv9 = Conv2D(n_filters,
                 3,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(ublock9)

    conv10 = Conv2D(num_classes, 1, padding='same')(conv9)

    model = tf.keras.Model(inputs=inputs, outputs=conv10)
    return model

# Remplacer par la forme réelle de vos images et le nombre de canaux (par exemple, (1440, 1440, 3) pour des images RGB)
model = create_unet_model((1440, 1440, 3), num_classes=12)
model.compile(optimizer='adam', 
              loss=SparseCategoricalCrossentropy(from_logits=True),  # Assuming softmax activation in the last layer
              metrics=['accuracy'])



In [6]:
model.fit(X_train, Y_train, batch_size=8, epochs=7, validation_data=(X_test, Y_test))
model.evaluate(X_test, Y_test)

Epoch 1/7
30/30 [==============================] - 10473s 350s/step - loss: 14.6337 - accuracy: 0.3171 - val_loss: 2.2316 - val_accuracy: 0.5020
Epoch 2/7
30/30 [==============================] - 10289s 344s/step - loss: 1.9662 - accuracy: 0.5055 - val_loss: 1.6904 - val_accuracy: 0.5477
Epoch 3/7
30/30 [==============================] - 10794s 362s/step - loss: 1.4751 - accuracy: 0.5280 - val_loss: 1.3193 - val_accuracy: 0.5891
Epoch 4/7
30/30 [==============================] - 10405s 348s/step - loss: 1.2046 - accuracy: 0.5899 - val_loss: 1.1842 - val_accuracy: 0.6103
Epoch 5/7
30/30 [==============================] - 11678s 390s/step - loss: 1.1388 - accuracy: 0.6175 - val_loss: 1.1690 - val_accuracy: 0.6388
Epoch 6/7
30/30 [==============================] - 11451s 384s/step - loss: 1.0495 - accuracy: 0.6504 - val_loss: 1.1238 - val_accuracy: 0.6387
Epoch 7/7
2/2 [==============================] - 744s 301s/step - loss: 1.1483 - accuracy: 0.6251


[1.1483187675476074, 0.6251271367073059]

In [8]:
from PIL import Image, ImageOps
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.metrics import SparseCategoricalCrossentropy

label_to_color = {v: k for k, v in color_to_label.items()}

# Charger et prétraiter l'image
def load_and_preprocess_image(image_path, target_size):
    image = load_img(image_path, target_size=target_size) # Assurez-vous que target_size correspond à la taille d'entrée du modèle
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0) # Ajouter une dimension pour le batch
    return image

def prediction_to_image(prediction):
    # Prendre l'indice de la classe la plus probable
    prediction = np.argmax(prediction, axis=-1)

    # Initialiser une image RGB vide
    colored_prediction = np.zeros((prediction.shape[0], prediction.shape[1], 3), dtype=np.uint8)

    # Appliquer la couleur à chaque pixel
    for label, color in label_to_color.items():
        colored_prediction[prediction == label] = color

    return Image.fromarray(colored_prediction)

# Faire une prédiction et afficher avec Pillow
def predict_and_display(model, image_path, target_size=(1440, 1440)):
    image = load_and_preprocess_image(image_path, target_size)
    prediction = model.predict(image)[0] # Récupérer la prédiction pour le premier élément du batch

    # Convertir la prédiction en image PIL et afficher
    original_image = Image.open(image_path)
    predicted_image = prediction_to_image(prediction)

    original_image.show(title='Image Originale')
    predicted_image.show(title='Prédiction du Modèle')

# Exemple d'utilisation
image_path = 'TrainData/Esa_tile2_Sat/ESA_WorldCover_10m_2021_V200_N36E027_Map.tif' # Remplacer par le chemin de votre image
predict_and_display(model, image_path)

1/1 [==============================] - 8s 8s/step


In [10]:
model.save("testmodel.h5")

In [13]:
from tensorflow.keras.models import load_model

model = load_model('testmodel.h5')
model.fit(X_train, Y_train, batch_size=8, epochs=3, validation_data=(X_test, Y_test))
model.evaluate(X_test, Y_test)
model.save("testmodelv2.h5")

Epoch 1/3
30/30 [==============================] - 11378s 377s/step - loss: 1.0422 - accuracy: 0.6500 - val_loss: 1.0522 - val_accuracy: 0.6441
Epoch 2/3
30/30 [==============================] - 11453s 380s/step - loss: 1.0609 - accuracy: 0.6491 - val_loss: 1.1029 - val_accuracy: 0.6257
Epoch 3/3
2/2 [==============================] - 744s 295s/step - loss: 1.0449 - accuracy: 0.6482


In [14]:
image_path = 'TrainData/Esa_tile2_Sat/ESA_WorldCover_10m_2021_V200_N36E027_Map.tif' # Remplacer par le chemin de votre image
predict_and_display(model, image_path)

1/1 [==============================] - 6s 6s/step
